# Odds_Forest

In [4]:
import pandas as pd
import numpy as np
import pickle
import random
from ALPHA_DT1 import *
import sys
sys.setrecursionlimit(10**9) 

Este módulo serve para extrair o Train/Test de um file de dados Alpha


In [2]:
def var_values(df,column): #dá nos uma lista dos valores de uma variável 
    #Nota, cada variável está associada a uma coluna
    coluna=list(df.iloc[:,column]) #transforma coluna em lista
    q=[]
    for val in coluna:
        if val not in q:
            q+=[val]
    return q

######################## Algoritmo de ordenação Mergesort
def fusao(u,v):
    res=[]
    i=0
    j=0
    for k in range(len(u)+len(v)):
        if i<len(u) and (j==len(v) or u[i]<v[j]):
            res.append(u[i])
            i=i+1
        else:
            res.append(v[j])
            j=j+1
    return res

def mergesort(w):
    if len(w)<2:
        return w
    else:
        m=len(w)//2
        w1=mergesort(w[:m])
        w2=mergesort(w[m:])
        return fusao(w1,w2)
#  #  #  #   #   #  #   #   #  #  #  #  #   # ALGORITMO DE ORDENAÇÃO MERGESORT



def is_numeric(x): #diz se x é número ou não
    return (isinstance(x,int)) or (isinstance(x,float)) or (isinstance(x,np.int64)) or (isinstance(x,np.float64))

#NOTAAA: Parece q os números do pandas estão em numpy.int64  ou numpy.float64


def string_column(df,column):
    #True se coluna constituída por strings
    q=var_values(df,column)
    l=len(q)
    i=0
    found=False
    while i<l and not(found):
        if not(isinstance(q[i],str)):
            found=True
        i+=1
    return not(found)
    

def is_categorical(df,column): 
    #diz se uma coluna numérica é categórica (sim/não, 0/1) (True)
    #ou se é variável contínua (False)
    if not(string_column(df,column)):
        return len(var_values(df,column))==2
    else:
        print('Esta coluna é de str, não mede esta função')
        return 'Esta coluna é de str, não mede esta função'
    
    
       
def not_categorical_values(df,column):
    #dá as médias dos valores adjacentes para fazer as perguntas
    v1=mergesort(var_values(df,column)) #dá jeito ter estes valores ordenados
    values=[]
    for i in range(len(v1)-1):
        values+=[((v1[i]+v1[i+1])/2)]
    return values 
 
    
    
def depend_counts(df):
    #print(df)
    #devolve um dicionário com os valores da variável dependente e as vezes q aparecem
    counts={}
    coluna=list(df.iloc[:,-1])
    for val in coluna:
        if val not in counts:
            counts[val]=0
        counts[val]+=1
    return counts

class Question:
    
    def __init__(self,column,value1,value2=None,tipo=0):
        self.column=column
        self.value1=value1
        self.value2=value2
        self.tipo=tipo
        
    def match(self,example):
        #verifica se o example dá True ou False à pergunta
        #example é uma linha/amostra em pd.DataFrame
        val=example.iloc[0][self.column] #como é uma linha, queremos 0
        if is_numeric(val):
            if self.tipo==0:
                return val > self.value1
            else:
                return self.value1<= val <= self.value2
        else:
            return val == self.value1

        
class Odd_Node:
    
    def __init__(self,question,true_branch,false_branch): #,tipo):
    #tipo serve para dizer se usasse os dois branch ou só um e qual
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch
        #self.ntipo=tipo

        
def proba(df):
    #calcula a probabilidade dos valores de um dicionário (em percentagem, para evitar float errors)
    #É necessário para as Leafs
    ind=len(df.index)
    dicti=depend_counts(df)
    new={0:0,1:0}  # este módulo de3 decision trees está feito para var dep de 0/1 APENAS
    for var in dicti:
        prob=round(100*(dicti[var]/ind),2)
        new[var]=prob
    return new    #devolve dicionário    
        
        
class Odd_Leaf:
    
    def __init__(self,df):
        self.prediction=depend_counts(df)
        self.probability=proba(df)
        self.df=df

class Odd_Forest:
#tipo floresta para o modelo das apostas
    def __init__(self):
        self.trees=[]
        
    def add_tree(self,tree):
        (self.trees)+=[tree]
        
    def size(self):
        return len(self.trees)
    
    def get_tree(self,ind):
        return self.trees[ind]
    
    def merge_forests(self,forest):
        if forest!=None:    
            (self.trees)+=(forest.trees)
            
def partition(df, question):
    #divide uma dataframe em valores que que acertam a pergunta e que erram
    true_rows,false_rows=pd.DataFrame(),pd.DataFrame()
    for i in range(len(df.index)):
        l=pd.DataFrame(df.iloc[i,:]).T #linha/amostra em linha
        if question.match(l):
            true_rows=pd.concat([true_rows,l],sort=False)
        else:
            false_rows=pd.concat([false_rows,l],sort=False)
    false_rows=false_rows.reset_index(drop=True)
    true_rows=true_rows.reset_index(drop=True)
    return true_rows,false_rows








    
    

def ohshit_aux(df,column):
    val=len(var_values(df,column))
    diff=[3]
    diff+=[1+(val%6)]
    mini = min(diff)
    return mini    
    
    
def column_questions(df,column,perguntas=1):
    #dá-nos a lista de todas as perguntas de uma coluna
    questions=[]
    if string_column(df,column):
        #print('gone to string')
        values = var_values(df,column)
        for val in values:
            q=Question(column,val)
            questions+=[q]
    else:
        if is_categorical(df,column):
            #print('gone to categorical')
            values = var_values(df,column) 
            assert len(values)==2
            value = (values[0]+values[1])/2
            q=Question(column,value,tipo=0)
            questions+=[q]
        else:
            #print('oh shit, here we go again, best_question')
            values=not_categorical_values(df,column)
            #print(values,'oh shit values')
            for i in range(perguntas): #Faz ciclo dos tipos de perguntas (1 ou 2)
                if i==0: #para perguntas tipo 0
                    for val in values:
                        q=Question(column,val,tipo=0)
                        questions+=[q]
                            
                else: #para perguntas tipo 1
                    mini=ohshit_aux(df,column)
                    for i1 in range(len(values)-mini): #se i1 for para lá de values[-2], não rende (ver caderno)
                        for i2 in range(i1+mini,len(values)-1):
                            q=Question(column,values[i1],values[i2],tipo=1)
                            questions+=[q]
    return questions


def all_questions(df):
    #devolve uma lista com todas as perguntas possíveis 
    Q=[]
    columns=len(df.columns)
    for i in range(columns-1): #não queremos a última pergunta
        Q+=column_questions(df,i)
    return Q

def best_partition(df,q,var): #diz-nos qual a partition com melhor real_pp (t(True) ou f(False))
    t,f=partition(df,q)
    if len(t)==0 or len(f)==0:
        #print('tadaaaaa,best_partition')
        return t,f,'skip'
    first_pp=proba(df)[var]
    if proba(t)[var]>=proba(f)[var]:
        return t,f,True
    elif proba(f)[var]>proba(t)[var]:
        return t,f, False
    else:
        print('erro, best_partition')
        return 'erro'


def indice(df,var,odd,original_df=None,big_first=False): #diz qual o índice da proba em relação à odd
    y=proba(df)[var] #real_pp/first_pp
    if big_first:
        p=proba(original_df)[var]
    else:
        p=round(100*(1/odd),2) #odd_percent
    #print(y,p,(y*((100-p)/p)))
    if y>99.99:
        res=5
    else:
        res=1-(np.log((100-y)/(y*((100-p)/p)))/2)
    return round(res,6),y #devolve índice e a proba


####### Funções para a FUNÇÃO LAMBDA ####  # #  # #  # # #  # # #  # #  # #
def a3(i,f,n):
    #m/q
    m=(2*f*i)-(4*f*n*i)+(i)-(2*f*n)
    q=(n*i)*((f**2)-(f*i))
    resA=m/q
    return resA

def b3(i,f,n):
    #m/q
    m=(4*i)+(2)-(a3(i,f,n)*(i**2))
    q=i
    resB=m/q
    return resB

def Lambda(i,f,n,ind):
    #m/q
    m=(2*(ind))+1
    q=(a3(i,f,n)*(ind**2))+(b3(i,f,n)*ind)
    resL=m/q
    return round(resL*100,3)
    
#  # #  # #  # # #  # # # #  # # #  # # #  # # #  # # # # #  #

def check_permutation(w1,w2):
    if len(w1)!=len(w2):
        return False
    else:
        dix1={}
        dix2={}
        for i in range(len(w1)):
            if w1[i] not in dix1:
                dix1[w1[i]]=0
            dix1[w1[i]]+=1
            if w2[i] not in dix2:
                dix2[w2[i]]=0
            dix2[w2[i]]+=1
        for n in dix2:
            if n not in dix1:
                return False
            if dix2[n]!=dix1[n]:
                return False
    return True
        

def question_check(proib,q_put):
    #este dá para fazer em miultiprocessing
    for lista in proib:
        if check_permutation(lista,q_put):
            return False
    return True

def list_master_aux1(q,q_1): 
    #q---> lista de perguntas
    #q_1--> lista em análise da q_list
    qw=q_1[0]
    i=0
    done=False
    while i<len(q) and not(done):
        if qw==q[i]:
            done=True
        else:
            i+=1
    return q[(i+1):]
#devolve a lista da perguntas a partir de uma pergunta qq
    

def list_master(w,i,check=None,q=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]):
    if check=='check':  
        if len(w)==i:
            if len(w)==0:
                w+=[q]
            else:
                if list_master_aux1(q,w[-1])!=[]:
                    w+=[list_master_aux1(q,w[-1])]   
    #if check=='restart':
    #    n=1
    #    done=False
    #    while n<=len(w) and not(done):
    #        if len(w[-n])>1:
    #            k=(-n)
    #            done=True
    #        n+=1
    #            
    #    if done: #não chegou ao fim
    #        w=w[:k]+[w[k][1:]]
    #    else: #chegou ao fim
    #        w=[[]]
            
    if check=='continue' or check=='restart':
        q_continue=w[:(i+1)]
        n=1
        done=False
        while n<=len(w) and not(done):
            if len(q_continue[-n])>1:
                k=(-n)
                done=True
            n+=1
                
        if done: #não chegou ao fim
            w=q_continue[:k]+[q_continue[k][1:]]
        else: #chegou ao fim
            w=[[]]
            
    if check=='restart':
        ind=0
    elif check=='check' and list_master_aux1(q,w[-1])==[] and w[0][0]==q[-1]:
        ind='breakall'
    elif check=='check' and list_master_aux1(q,w[-1])==[]:
        ind='break'
    else:
        ind=len(w)-1
    #print(w,ind)
    return w,ind #também dá return do índice

def size_percent(original_df,df):
    size1=len(original_df.index)
    size2=len(df.index)
    resp=round(100*(size2/size1),2)
    return resp


def Magic3(original_df,df,questions,odd,var,forest,q_list=[],ind=0,bl=None,L_limit=1.2,L_min_perc=0.15):
    #L_limit é o índice até ao qual vamos no lamda
    
    first_pp=proba(original_df)[var] #podemos por isto como parâmetro
    odd_percent=round(100*(1/odd),2) #podemos por isto como parâmetro
    i_first,lixo_i=indice(original_df,var,odd)  #podemos por isto como parâmetro
    print('passo------------>',ind)
    
    if q_list==[[]]:
        print('katchau')
        return None
    
    
    if bl==False or bl==True:
        if bl==False:
            print('Leafing False')
        else:
            if first_pp>=odd_percent:
                teste_ind, teste_percentage=indice(df,var,odd,original_df=original_df,big_first=True)
            else:
                teste_ind, teste_percentage=indice(df,var,odd)
            true_size_perc=size_percent(original_df,df)
            
            print('Leafing True')
            print('Le indice',teste_ind,'(Le percentage',teste_percentage,'%) Le size Percent',true_size_perc)
        return Odd_Leaf(df)          
   
    else:
        q_list,lixo=list_master(q_list,ind,'check',q=questions) #aqui não precisamos do índice, por isso é lixo
        print('Le size du q_list ==',len(q_list))
        print('Le current size du q_list',len(q_list[ind]))
        print('Le size du [0]==',len(q_list[0]))
        print('Le LAST size du q_list==',len(q_list[-1]),'__([-1])')
        if lixo=='break':
            print('sera que funciona?') ##########
            return Odd_Leaf(df)######################
        elif lixo=='breakall':
            print('---------',forest.size(),'---------')
            print('La derniere floreste',forest)
            print('indiceeeee',ind)
            return None
        q=q_list[ind][0]
        t,f,b=best_partition(df,q,var)
        if len(t)==0 or len(f)==0:
            if len(q_list[ind])==1: #new
                return Odd_Leaf(df)  #new
            else:
                print('continuando........','Len=0')
                q_list,ind=list_master(q_list,ind,'continue',q=questions)
                return Magic3(original_df,df,questions,odd,var,forest=forest,q_list=q_list,ind=ind,bl=bl,L_limit=L_limit,L_min_perc=L_min_perc)
            #continue #neste continue continuamos com df e não pre_df
        if b:
            print('gone b')
            if first_pp>=odd_percent:
                T_ind, lixoT=indice(t,var,odd,original_df=original_df,big_first=True)
            else:
                T_ind, lixoT=indice(t,var,odd)
            
            T_percentage=size_percent(original_df,t)
            
            print('Indicadores: Indice:',T_ind,' proba:',lixoT,' size_percent:',T_percentage)
            
            if T_ind<=L_limit: #necessita de lambda
                if T_percentage>=Lambda(i_first,L_limit,L_min_perc,T_ind): #let's keep it
                    print('Gooooooooood Lambdaaaaaa')
                    true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl='continue2',L_limit=L_limit,L_min_perc=L_min_perc)
                    false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)
                else:
                    if len(q_list[ind])==1:  #new
                        print('continuando........','Lamda, OMFGGGGGGGGGGGGGGG')
                        return Odd_Leaf(df) #new
                    else:
                        print('continuando........','Lamda')
                        q_list,ind=list_master(q_list,ind,'continue',q=questions)
                        return Magic3(original_df,df,questions,odd,var,forest=forest,q_list=q_list,ind=ind,bl=bl,L_limit=L_limit,L_min_perc=L_min_perc)
                        #continue #proba demasiado baixa para usar 

            else:
                if T_percentage<10:
                    if len(q_list[ind])==1:  #new
                        print('continuando........','<0.02, OMFGGGGGGGGGGGGGGG')
                        return Odd_Leaf(df) #new
                    else:
                        print('continuando........','<0.02')
                        q_list,ind=list_master(q_list,ind,'continue',q=questions)
                        return Magic3(original_df,df,questions,odd,var,forest=forest,q_list=q_list,ind=ind,bl=bl,L_limit=L_limit,L_min_perc=L_min_perc)
                        #continue #não queremos esta folha (proba<0.02)
                elif T_percentage>20 and T_ind<=2:
                    if len(q_list[ind])==1:  #new
                        print('>1, But not Folha Yet, OMFGGGGGGGGGGGGGGGGGG')
                        true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=True,L_limit=L_limit,L_min_perc=L_min_perc)
                        false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)#new
                    else:
                        print('>1, But not Folha Yet')
                        true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl='continue2',L_limit=L_limit,L_min_perc=L_min_perc)
                        false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)
                elif T_percentage<=20 or T_ind>2:
                    print('Here comes a Folhaaaaaaa')
                    true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=True,L_limit=L_limit,L_min_perc=L_min_perc)
                    false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)
                    

        else:
            print('gone else b')
            if first_pp>=odd_percent:
                F_ind, lixoF=indice(f,var,odd,original_df=original_df,big_first=True)
            else:
                F_ind, lixoF=indice(f,var,odd)
                
            F_percentage=size_percent(original_df,f)
            
            print('Indicadores: Indice:',F_ind,' proba:',lixoF,' size_percent:',F_percentage)
                
            if F_ind<=L_limit: #necessita de lambda
                print('<=1')
                if F_percentage>=Lambda(i_first,L_limit,L_min_perc,F_ind): #let's keep it
                    print('Gooooooooood Lambdaaaaaa')
                    true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)
                    false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl='continue2',L_limit=L_limit,L_min_perc=L_min_perc)
                else:
                    if len(q_list[ind])==1:  #new
                        print('continuando........','Lamda, OMFGGGGGGGGGGGGGGG')
                        return Odd_Leaf(df) #new
                    else:
                        print('continuando........','Lamda')
                        q_list,ind=list_master(q_list,ind,'continue',q=questions)
                        return Magic3(original_df,df,questions,odd,var,forest=forest,q_list=q_list,ind=ind,bl=bl,L_limit=L_limit,L_min_perc=L_min_perc)
                    #continue #proba demasiado baixa para usar 

            else:
                print('else >1')
                if F_percentage<10:
                    if len(q_list[ind])==1:  #new
                        print('continuando........','<0.02, OMFGGGGGGGGGGGGGGG')
                        return Odd_Leaf(df) #new
                    else:
                        print('continuando........','<0.02')
                        q_list,ind=list_master(q_list,ind,'continue',q=questions)
                        return Magic3(original_df,df,questions,odd,var,forest=forest,q_list=q_list,ind=ind,bl=bl,L_limit=L_limit,L_min_perc=L_min_perc)
                    #continue #não queremos esta folha (proba<0.02)
                elif F_percentage>20 and F_ind<=2:
                    if len(q_list[ind])==1:  #new
                        print('>1, But not Folha Yet, OMFGGGGGGGGGGGGGGGGGG')
                        true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)
                        false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=True,L_limit=L_limit,L_min_perc=L_min_perc) #new
                    else:
                        print('>1, But not Folha Yet')
                        true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)
                        false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl='continue2',L_limit=L_limit,L_min_perc=L_min_perc)
                elif F_percentage<=20 or F_ind>2:
                    print('Here comes a Folhaaaaaaa')
                    true_branch=Magic3(original_df,t,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=False,L_limit=L_limit,L_min_perc=L_min_perc)
                    false_branch=Magic3(original_df,f,questions,odd,var,forest=forest,q_list=q_list,ind=ind+1,bl=True,L_limit=L_limit,L_min_perc=L_min_perc)
        if ind>=1:
            print('Juntando Odd_Nodes')
            print('Dados:',b)
            return Odd_Node(q,true_branch,false_branch)
        else:
            print('---------',forest.size(),'---------')
            q_list,ind=list_master(q_list,ind,'restart',q=questions)
            if q_list==[[]]:
                return forest
            else:
                forest.add_tree(Odd_Node(q,true_branch,false_branch))
                print('La florestaaaaa',forest)
                return forest.merge_forests(Magic3(original_df,original_df,questions,odd,var,forest=forest,q_list=q_list,ind=ind,bl=None,L_limit=L_limit,L_min_perc=L_min_perc))
            
            
def classify_row(df_test,node,probability=True):
    #escrevi df_test, mas isto é para apenas uma linha
    if isinstance(node,Odd_Leaf):
        if probability:
            return node.probability
        else:
            return node.predictions
    else:
        if node.question.match(df_test):
            return classify_row(df_test,node.true_branch,probability)
        else:
            return classify_row(df_test,node.false_branch,probability)
        
def tree_1D(row,tree,odd,var,probability=True):
    #maneira (d) do primeiro passo das previsões
    dix=classify_row(row,tree,probability)
    pred=dix[var]
    limit=round(100*(1/odd),2)
    print(pred,'pred')
    if pred<=limit:
        return 0
    else:
        return round(100*((pred/limit)-1),2)
    

def row_Forest(row,forest,odd,var,probability=True):
    #devolve a lista de previsões de uma linha de uma floresta
    sizeF=forest.size()
    pred_list=[]
    for t in range(sizeF):
        tree=forest.get_tree(t)
        val=tree_1D(row,tree,odd,var,probability)
        pred_list+=[val]
    return pred_list

def Forest_2A(row,forest,odd,var,probability=True):
    #devolve a previsão de uma linha segundo o método (a) do segundo passo
    "por fazer"
    
    
    
    
####### aux do find_best -----------
def find_best_aux(node,original_df,current_df,var,questions=[]):
    #questions e probas são as listas que vão ter as perguntas e as probas respetivamente
    #dps, no final juntamos as duas e ficamos com a lista pretendida
    #a representação da lista está na função find_best
    q=node.question
    t=node.true_branch
    f=node.false_branch
    true_df,false_df=partition(current_df,q)
    if type(t)==Odd_Leaf and type(f)==Odd_Leaf:
        if proba(true_rows)[var]>=proba(false_rows)[var]:
            le_size=round(100*((len(true_rows.index))/(len(original_df.index))),2)
            probaz=proba(true_rows)[var]
            questions+=[q,True]
            return [questions,[le_size,probaz]]
        else:
            le_size=round(100*((len(false_rows.index))/(len(original_df.index))),2)
            probaz=proba(false_rows)[var]
            questions+=[q,False]
            return [questions,[le_size,probaz]]
    elif type(t)==Odd_Leaf and type(f)!=Odd_Leaf:
        questions+=[[q,False]]
        res=find_best_aux(f,original_df,false_df,var,questions)
    elif type(t)!=Odd_Leaf and type(f)==Odd_Leaf:
        questions+=[[q,True]]
        res=find_best_aux(t,original_df,true_df,var,questions)
    else:
        print('WTF')
        
    return res
        
#------------------------------------#
    
def find_best(forest,var):
    #encontra a melhor folha de cada árvore e devolve uma 'lista', para a conseguirmos encontrar
    #[[[pergunta,T/F],[pergunta,T/F],......],[Size_perc,real_pp]]
    #assim podíamos testar nas folhas de excel dos varios anos
    sizeF=forest.size()
    resu=[]
    for t in range(sizeF):
        tree=forest.get_tree(t)
        ma=find_best_aux(tree,original_df,original_df,var,questions=[])
        resu+=[ma]
    return resu
        
        
        
        
    
        
        
    
            
            
#def get_predictions1(df_test,forest): #dá-nos as previsões da maneira 1
    

In [3]:
prem15=odd_to_file('PremAlpha06_20',1.5)
prem15=prepare_df(prem15)
prem15=round_odd(prem15,1.5)
prem15_Train, prem15_Test=train_test_split(prem15,test_percentage=10)

338 size
33 test_size
305
134 167


In [4]:
qw2=all_questions(prem15)
len(qw2)

258

In [10]:
f=Odd_Forest()

In [6]:
isinstance(f,Odd_Forest)

True

In [11]:
Magic3(prem15_Train,prem15_Train,qw2,1.5,1,f,q_list=[],ind=0,bl=None,L_limit=1.3,L_min_perc=0.15)

passo------------> 0
Le size du q_list == 1
Le current size du q_list 258
Le size du [0]== 258
Le LAST size du q_list== 258 __([-1])
gone b
Indicadores: Indice: 0.969478  proba: 65.3  size_percent: 71.8
Gooooooooood Lambdaaaaaa
passo------------> 1
Le size du q_list == 2
Le current size du q_list 257
Le size du [0]== 258
Le LAST size du q_list== 257 __([-1])
gone b
Indicadores: Indice: 1.202658  proba: 75.0  size_percent: 14.43
continuando........ Lamda
passo------------> 1
Le size du q_list == 2
Le current size du q_list 256
Le size du [0]== 258
Le LAST size du q_list== 256 __([-1])
continuando........ Len=0
passo------------> 1
Le size du q_list == 2
Le current size du q_list 255
Le size du [0]== 258
Le LAST size du q_list== 255 __([-1])
gone else b
Indicadores: Indice: 1.008801  proba: 67.06  size_percent: 55.74
<=1
Gooooooooood Lambdaaaaaa
passo------------> 2
Leafing False
passo------------> 2
Le size du q_list == 3
Le current size du q_list 254
Le size du [0]== 258
Le LAST size d

gone else b
Indicadores: Indice: 1.482465  proba: 84.0  size_percent: 8.2
else >1
continuando........ <0.02
passo------------> 6
Le size du q_list == 7
Le current size du q_list 222
Le size du [0]== 258
Le LAST size du q_list== 222 __([-1])
gone else b
Indicadores: Indice: 1.077  proba: 70.0  size_percent: 13.11
<=1
continuando........ Lamda
passo------------> 6
Le size du q_list == 7
Le current size du q_list 221
Le size du [0]== 258
Le LAST size du q_list== 221 __([-1])
gone b
Indicadores: Indice: 1.105666  proba: 71.19  size_percent: 19.34
continuando........ Lamda
passo------------> 6
Le size du q_list == 7
Le current size du q_list 220
Le size du [0]== 258
Le LAST size du q_list== 220 __([-1])
gone b
Indicadores: Indice: 1.173999  proba: 73.91  size_percent: 15.08
continuando........ Lamda
passo------------> 6
Le size du q_list == 7
Le current size du q_list 219
Le size du [0]== 258
Le LAST size du q_list== 219 __([-1])
gone b
Indicadores: Indice: 1.077  proba: 70.0  size_percent:

gone else b
Indicadores: Indice: 1.320953  proba: 79.17  size_percent: 7.87
else >1
continuando........ <0.02
passo------------> 15
Le size du q_list == 16
Le current size du q_list 181
Le size du [0]== 258
Le LAST size du q_list== 181 __([-1])
gone b
Indicadores: Indice: 1.394105  proba: 81.48  size_percent: 8.85
continuando........ <0.02
passo------------> 15
Le size du q_list == 16
Le current size du q_list 180
Le size du [0]== 258
Le LAST size du q_list== 180 __([-1])
gone b
Indicadores: Indice: 1.423473  proba: 82.35  size_percent: 5.57
continuando........ <0.02
passo------------> 15
Le size du q_list == 16
Le current size du q_list 179
Le size du [0]== 258
Le LAST size du q_list== 179 __([-1])
gone b
Indicadores: Indice: 1.45795  proba: 83.33  size_percent: 3.93
continuando........ <0.02
passo------------> 15
Le size du q_list == 16
Le current size du q_list 178
Le size du [0]== 258
Le LAST size du q_list== 178 __([-1])
gone b
Indicadores: Indice: 1.549056  proba: 85.71  size_per

gone else b
Indicadores: Indice: 5  proba: 100.0  size_percent: 0.66
else >1
continuando........ <0.02
passo------------> 8
Le size du q_list == 9
Le current size du q_list 230
Le size du [0]== 255
Le LAST size du q_list== 230 __([-1])
gone else b
Indicadores: Indice: 5  proba: 100.0  size_percent: 0.66
else >1
continuando........ <0.02
passo------------> 8
Le size du q_list == 9
Le current size du q_list 229
Le size du [0]== 255
Le LAST size du q_list== 229 __([-1])
gone else b
Indicadores: Indice: 5  proba: 100.0  size_percent: 0.66
else >1
continuando........ <0.02
passo------------> 8
Le size du q_list == 9
Le current size du q_list 228
Le size du [0]== 255
Le LAST size du q_list== 228 __([-1])
gone else b
Indicadores: Indice: 5  proba: 100.0  size_percent: 0.66
else >1
continuando........ <0.02
passo------------> 8
Le size du q_list == 9
Le current size du q_list 227
Le size du [0]== 255
Le LAST size du q_list== 227 __([-1])
gone else b
Indicadores: Indice: 5  proba: 100.0  size_p

gone b
Indicadores: Indice: 5  proba: 100.0  size_percent: 0.66
continuando........ <0.02
passo------------> 9
Le size du q_list == 10
Le current size du q_list 193
Le size du [0]== 255
Le LAST size du q_list== 193 __([-1])
gone b
Indicadores: Indice: 5  proba: 100.0  size_percent: 0.66
continuando........ <0.02
passo------------> 9
Le size du q_list == 10
Le current size du q_list 192
Le size du [0]== 255
Le LAST size du q_list== 192 __([-1])
gone b
Indicadores: Indice: 5  proba: 100.0  size_percent: 0.33
continuando........ <0.02
passo------------> 9
Le size du q_list == 10
Le current size du q_list 191
Le size du [0]== 255
Le LAST size du q_list== 191 __([-1])
continuando........ Len=0
passo------------> 9
Le size du q_list == 10
Le current size du q_list 190
Le size du [0]== 255
Le LAST size du q_list== 190 __([-1])
gone b
Indicadores: Indice: 1.135797  proba: 72.41  size_percent: 28.52
Gooooooooood Lambdaaaaaa
passo------------> 10
Le size du q_list == 11
Le current size du q_list

...

gone else b
Indicadores: Indice: 1.30295  proba: 78.57  size_percent: 13.77
else >1
Here comes a Folhaaaaaaa
passo------------> 4
Leafing False
passo------------> 4
Leafing True
Le indice 1.30295 (Le percentage 78.57 %) Le size Percent 13.77
Juntando Odd_Nodes
Dados: False
passo------------> 3
Leafing False
Juntando Odd_Nodes
Dados: True
Juntando Odd_Nodes
Dados: False
passo------------> 1
Leafing False
--------- 106 ---------
La florestaaaaa <__main__.Odd_Forest object at 0x000001C4DA8276D8>
passo------------> 0
Le size du q_list == 1
Le current size du q_list 27
Le size du [0]== 27
Le LAST size du q_list== 27 __([-1])
gone b
Indicadores: Indice: 0.982105  proba: 65.87  size_percent: 82.62
Gooooooooood Lambdaaaaaa
passo------------> 1
Le size du q_list == 2
Le current size du q_list 26
Le size du [0]== 27
Le LAST size du q_list== 26 __([-1])
gone else b
Indicadores: Indice: 1.520652  proba: 85.0  size_percent: 6.56
else >1
continuando........ <0.02
passo------------> 1
Le size du q_li

gone else b
Indicadores: Indice: 1.895967  proba: 92.31  size_percent: 4.26
else >1
continuando........ <0.02
passo------------> 1
Le size du q_list == 2
Le current size du q_list 15
Le size du [0]== 24
Le LAST size du q_list== 15 __([-1])
gone b
Indicadores: Indice: 1.058798  proba: 69.23  size_percent: 29.84
continuando........ Lamda
passo------------> 1
Le size du q_list == 2
Le current size du q_list 14
Le size du [0]== 24
Le LAST size du q_list== 14 __([-1])
gone else b
Indicadores: Indice: 1.071536  proba: 69.77  size_percent: 14.1
<=1
continuando........ Lamda
passo------------> 1
Le size du q_list == 2
Le current size du q_list 13
Le size du [0]== 24
Le LAST size du q_list== 13 __([-1])
gone else b
Indicadores: Indice: 1.140562  proba: 72.6  size_percent: 23.93
<=1
Gooooooooood Lambdaaaaaa
passo------------> 2
Leafing False
passo------------> 2
Le size du q_list == 3
Le current size du q_list 12
Le size du [0]== 24
Le LAST size du q_list== 12 __([-1])
continuando........ Len=0


gone b
Indicadores: Indice: 1.0  proba: 66.67  size_percent: 75.74
Gooooooooood Lambdaaaaaa
passo------------> 2
Le size du q_list == 3
Le current size du q_list 16
Le size du [0]== 22
Le LAST size du q_list== 16 __([-1])
gone else b
Indicadores: Indice: 1.505903  proba: 84.62  size_percent: 4.26
else >1
continuando........ <0.02
passo------------> 2
Le size du q_list == 3
Le current size du q_list 15
Le size du [0]== 22
Le LAST size du q_list== 15 __([-1])
gone else b
Indicadores: Indice: 1.069878  proba: 69.7  size_percent: 21.64
<=1
continuando........ Lamda
passo------------> 2
Le size du q_list == 3
Le current size du q_list 14
Le size du [0]== 22
Le LAST size du q_list== 14 __([-1])
gone else b
Indicadores: Indice: 1.080577  proba: 70.15  size_percent: 21.97
<=1
continuando........ Lamda
passo------------> 2
Le size du q_list == 3
Le current size du q_list 13
Le size du [0]== 22
Le LAST size du q_list== 13 __([-1])
gone else b
Indicadores: Indice: 1.057391  proba: 69.17  size_per

gone else b
Indicadores: Indice: 1.0  proba: 66.67  size_percent: 6.89
<=1
continuando........ Lamda
passo------------> 1
Le size du q_list == 2
Le current size du q_list 15
Le size du [0]== 19
Le LAST size du q_list== 15 __([-1])
gone else b
Indicadores: Indice: 1.008801  proba: 67.06  size_percent: 27.87
<=1
continuando........ Lamda
passo------------> 1
Le size du q_list == 2
Le current size du q_list 14
Le size du [0]== 19
Le LAST size du q_list== 14 __([-1])
gone else b
Indicadores: Indice: 1.017428  proba: 67.44  size_percent: 28.2
<=1
continuando........ Lamda
passo------------> 1
Le size du q_list == 2
Le current size du q_list 13
Le size du [0]== 19
Le LAST size du q_list== 13 __([-1])
gone else b
Indicadores: Indice: 1.014926  proba: 67.33  size_percent: 49.18
<=1
Gooooooooood Lambdaaaaaa
passo------------> 2
Leafing False
passo------------> 2
Le size du q_list == 3
Le current size du q_list 12
Le size du [0]== 19
Le LAST size du q_list== 12 __([-1])
continuando........ Len=0

gone else b
Indicadores: Indice: 1.029778  proba: 67.98  size_percent: 74.75
<=1
Gooooooooood Lambdaaaaaa
passo------------> 1
Leafing False
passo------------> 1
Le size du q_list == 2
Le current size du q_list 10
Le size du [0]== 11
Le LAST size du q_list== 10 __([-1])
continuando........ Len=0
passo------------> 1
Le size du q_list == 2
Le current size du q_list 9
Le size du [0]== 11
Le LAST size du q_list== 9 __([-1])
continuando........ Len=0
passo------------> 1
Le size du q_list == 2
Le current size du q_list 8
Le size du [0]== 11
Le LAST size du q_list== 8 __([-1])
continuando........ Len=0
passo------------> 1
Le size du q_list == 2
Le current size du q_list 7
Le size du [0]== 11
Le LAST size du q_list== 7 __([-1])
continuando........ Len=0
passo------------> 1
Le size du q_list == 2
Le current size du q_list 6
Le size du [0]== 11
Le LAST size du q_list== 6 __([-1])
continuando........ Len=0
passo------------> 1
Le size du q_list == 2
Le current size du q_list 5
Le size du [0]=

# -------------------------------------------------------------------------------------------------------------

In [5]:
read=open('Odd_Forest_15.pickle','rb')
forest=pickle.load(read)
train=pickle.load(read)
test=pickle.load(read)


ModuleNotFoundError: No module named 'Odd_Forest1'

In [8]:
f

In [9]:
f

In [13]:
type(f)
isinstance(f,Odd_Forest)

False

In [11]:
l=tree.true_branch
type(l)

NameError: name 'tree' is not defined

In [28]:
isinstance(tree,Odd_Node)

False

In [23]:
isinstance(l,Odd_Leaf)

False

In [12]:
row=pd.DataFrame(prem15_Test.iloc[0,:]).T

row_Forest(row,f,1.5,1,probability=True)

AttributeError: 'Odd_Leaf' object has no attribute 'question'

In [30]:
def a1(i):
    res1=((2-(26*i))/((i**2)-i))
    return round(res1,6)

def b1(i):
    res2=(((30*(i**2))-(4*i)-2)/((i**2)-i))
    return round(res2,6)

def Lambda(i_partition,i_first): 
#i_partition -> índice da percentagem da partition em análise (a melhor partition)
#i_first -> índice da percentagem do first_pp
    #print((a1(i_first)))
    #print((b1(i_first)))
    res3=(((2*(i_partition)+1)/(((a1(i_first))*(i_partition**2))+((b1(i_first))*(i_partition)))))
    return round(res3*100,6)

In [44]:
def a2(i,f):
    #m/n
    m=(16*f*i)+(10*i)-(2*f)
    n=i*((f**2)-(i*f))
    resA=m/n
    return resA

def b2(i,f):
    #m/n
    aux=(f**2)-(i*f)
    m=(4*i*aux)+(2*aux)-(16*f*(i**2))-(10*(i**2))+(2*f*i)
    n=i*aux
    resB=m/n
    return resB

def Lambda1(i,f,ind):
    #m/n
    m=(2*ind)+1
    n=((ind**2)*a2(i,f))+(b2(i,f)*ind)
    resL=m/n
    return round(resL,3)

In [9]:
def a3(i,f,n):
    #m/q
    m=(2*f*i)-(4*f*n*i)+(i)-(2*f*n)
    q=(n*i)*((f**2)-(f*i))
    resA=m/q
    return resA

def b3(i,f,n):
    #m/q
    m=(4*i)+(2)-(a3(i,f,n)*(i**2))
    q=i
    resB=m/q
    return resB

def Lambda2(i,f,n,ind):
    #m/q
    m=(2*(ind))+1
    q=(a3(i,f,n)*(ind**2))+(b3(i,f,n)*ind)
    resL=m/q
    return round(resL*100,3)
    
    
    

In [20]:
odd=1.7
ind=1/1.7
ind

0.5882352941176471

In [12]:
Lambda2(0.9126,1,0.2,1)

20.0

0.117

In [39]:
def list_master(w,i,check=None,q=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]):
    if check=='check':  
        if len(w)==i:
            if len(w)==0:
                w+=[q]
            else:
                w+=[list_master_aux1(q,w[-1])]   
    #if check=='restart':
    #    n=1
    #    done=False
    #    while n<=len(w) and not(done):
    #        if len(w[-n])>1:
    #            k=(-n)
    #            done=True
    #        n+=1
    #            
    #    if done: #não chegou ao fim
    #        w=w[:k]+[w[k][1:]]
    #    else: #chegou ao fim
    #        w=[[]]
            
    if check=='continue' or check=='restart':
        q_continue=w[:(i+1)]
        n=1
        done=False
        while n<=len(w) and not(done):
            if len(q_continue[-n])>1:
                k=(-n)
                done=True
            n+=1
                
        if done: #não chegou ao fim
            w=q_continue[:k]+[q_continue[k][1:]]
        else: #chegou ao fim
            w=[[]]
            
    if check=='restart':
        ind=0
    else:
        ind=len(w)-1
    print(w,ind)
    return w,ind #também dá return do índice

In [45]:
w,lixo=list_master([[1,2],[4,5]],2,'check')

[[1, 2], [4, 5], [5, 6, 7, 8, 9]] 2


In [13]:
list_master([[1,2],[3,4,5],[7,8,9,10,11],[15,16,17,18]],1,'continue')

[[1, 2], [4, 5]] 1


([[1, 2], [4, 5]], 1)

In [64]:
def list_master(w,i,check=None,q=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]):
    if check=='check':  
        if len(w)==i:
            if len(w)==0:
                w+=[q]
            else:
                w+=list_master_aux1(q)    
    if check=='restart' or check=='continue':
        i=1
        done=False
        while i<=len(w) and not(done):
            if len(w[-i])>1:
                k=(-i)
                done=True
            i+=1
                
        if done: #não chegou ao fim
            w=w[:k]+[w[k][1:]]
        else: #chegou ao fim
            w=[[]]
    if check=='restart':
        ind=0
    else:
        
        ind=i
    print(w,ind)
    return w,ind #também dá return do índice

In [41]:
w,ind=list_master([[1,2],[3,4,5],[7,8,9,10,11],[15,16,17,18]],2,'restart')

[[1, 2], [3, 4, 5], [8, 9, 10, 11]] 0


In [37]:
w,ind=list_master(w,2,'continue')

[[2]] 0


In [28]:
w,ind=list_master(w,ind+1,'check')

[[1, 2], [3, 4, 5], [7, 8, 9, 10, 11], [16, 17, 18]] 3
